In [1]:
import requests
import csv
import xlrd

In [33]:
def getFootballUrl(league,season,format = "csv"):
    return f"https://www.football-data.co.uk/mmz4281/{season}/{league}.{format}"

def downloadData(league, season):
    season = f"{season}{season+1}"
    url = getFootballUrl(league, season)
    print(url)
    r = requests.get(url)
    if(r.status_code == 200):
        with open(f'../data/{league}-{season}.csv', 'wb') as f:
            f.write(r.content)
    else:
        print("File not found")

def downloadAllEuropeanLeagues(startingYear):
    seasonShort = startingYear%100
    season = f"{seasonShort}{(seasonShort+1)%100}"
    url = getFootballUrl(f"all-euro-data-{startingYear}-{startingYear+1}",season,"xls")
    print(url)
    r = requests.get(url)
    firstCheck = 0 #TEMP
    if(r.status_code == 200):
        with open(f'../data/AllLeagues-{season}.xls', 'wb') as f:
            f.write(r.content)
        book = xlrd.open_workbook(f'../data/AllLeagues-{season}.xls')
        for leagueName in book.sheet_names():
            if leagueName == "G1": #Problem with greek league
                continue
            sh = book.sheet_by_name(leagueName)
            csvFile = open(f'../data/{leagueName}-{season}.csv', 'w')
            wr = csv.writer(csvFile)
            for rownum in range(sh.nrows):
                if firstCheck <= 2:
                    print(sh.row_values(rownum))
                    firstCheck += 1
                wr.writerow(sh.row_values(rownum))
            csvFile.close()
    else:
        print(f"Invalid URL: {url}")

In [40]:
downloadAllEuropeanLeagues(2019)

https://www.football-data.co.uk/mmz4281/1920/all-euro-data-2019-2020.xls


XLRDError: Excel xlsx file; not supported

In [20]:
downloadData('E0',19)

https://www.football-data.co.uk/mmz4281/1920/E0.csv


In [3]:
with open("../data/E0-1920.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['HomeTeam'] == 'Liverpool':
            
            print(f"{row['HomeTeam']} {row['FTHG']}:{row['FTAG']} {row['AwayTeam']}")

Liverpool 4:1 Norwich
Liverpool 3:1 Arsenal
Liverpool 3:1 Newcastle
Liverpool 2:1 Leicester
Liverpool 2:1 Tottenham
Liverpool 3:1 Man City
Liverpool 2:1 Brighton
Liverpool 5:2 Everton
Liverpool 2:0 Watford
Liverpool 1:0 Wolves
Liverpool 2:0 Sheffield United
Liverpool 2:0 Man United
Liverpool 4:0 Southampton
Liverpool 3:2 West Ham
Liverpool 2:1 Bournemouth
Liverpool 4:0 Crystal Palace
Liverpool 2:0 Aston Villa
Liverpool 1:1 Burnley
Liverpool 5:3 Chelsea
